In [1]:
from __future__ import print_function
import torch

from gklearn.utils.graphfiles import loadDataset
import networkx as nx
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np



In [2]:
def label_to_color(label):
    if label == 'C':
        return 0.1
    elif label == 'O':
        return 0.8
    
def nodes_to_color_sequence(G):
    return [label_to_color(c[1]['label'][0]) for c in G.nodes(data=True)]

Gs,y = loadDataset('/home/luc/TRAVAIL/DeepGED/Acyclic/trainset_0.ds')
    #'/data/chercheurs/brunl01/Acyclic/trainset_0.ds')
#for e in Gs[13].edges():
#    print(Gs[13][e[0]][e[1]]['bond_type'])
print('edge max label',max(max([[G[e[0]][e[1]]['bond_type'] for e in G.edges()] for G in Gs])))
G1 = Gs[13]
G2 = Gs[23]
print(y[13],y[23])
nx.draw_networkx(G1,with_labels=True,node_color = nodes_to_color_sequence(G1),cmap='autumn')
plt.figure()

nx.draw_networkx(G2,with_labels=True,node_color = nodes_to_color_sequence(G2),cmap='autumn')


#print(nx.to_dict_of_lists(Gs[13]))

# A vérifier: L'ordre des sommets de numpy_matrix(G) est il le même que nx.nodes(G)
def from_networkx_to_tensor(G,dict):
    A=nx.to_numpy_matrix(G)
    lab=[dict[G.nodes[v]['label'][0]] for v in nx.nodes(G)]
    return (torch.tensor(A).view(1,A.shape[0]*A.shape[1]),torch.tensor(lab))

edge max label 1
64.4 52.5


In [3]:
import gcn
import svd

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import svd
import gcn
#from svd import iterated_power as compute_major_axis
compute_major_axis=svd.CustomMajorAxis.apply
#torch.autograd.set_detect_anomaly(True)

class Net(nn.Module):
    
        
    def __init__(self,GraphList,normalize=False,device=torch.device("cpu")):
        super(Net, self).__init__()   
        self.normalize=normalize
        self.total_nb_graphs=len(GraphList)
        self.nb_feature=20
        
        dict,self.nb_edge_labels=self.build_node_dictionnary(GraphList)
        self.nb_labels=len(dict)
        print('nb_edge labels=',self.nb_edge_labels)
        self.device=device
        self.gcn=gcn.GCN2(self.nb_labels,self.nb_feature,dropout=0.1).to(self.device)               
        nb_edge_pair_label=int(self.nb_edge_labels*(self.nb_edge_labels-1)/2)
        
        self.node_weighs=nn.Parameter(torch.tensor(1.0/(nb_edge_pair_label+2.0))+(1e-3)*torch.rand(2,requires_grad=True,device=self.device)) # all substitution costs+ nodeIns/Del. old version: 0 node subs, 1 nodeIns/Del, 2 : edgeSubs, 3 edgeIns/Del
            
        self.edge_weighs=nn.Parameter(torch.tensor(1.0/(nb_edge_pair_label+2.0))+(1e-3)*torch.rand(nb_edge_pair_label+1,requires_grad=True,device=self.device)) #edgeIns/Del
        
        self.card=torch.tensor([G.order() for G in GraphList]).to(self.device)
        card_max=self.card.max()
        self.A=torch.empty((len(GraphList),card_max*card_max),dtype=torch.float,device=self.device)
        self.labels=torch.empty(self.total_nb_graphs,card_max,self.nb_labels,dtype=torch.float,device=self.device)
        print(self.A.shape)
        for k in range(len(GraphList)):
            A,l=self.from_networkx_to_tensor(GraphList[k],dict,card_max)             
            self.A[k,0:A.shape[1]]=A[0]
            self.labels[k,:,:]=l
        print('adjacency matrices',self.A)
        print('node labels',self.labels)
        print('order of the graphs',self.card)
        print('nb_edge_pair_label,nb_edge_labels',nb_edge_pair_label,self.nb_edge_labels)
        
    def forward(self, input):        
        ged=torch.zeros(len(input)).to(self.device) 
        node_subst,nodeInsDel,edge_costs,edgeInsDel=self.from_weighs_to_costs()
        
        f_computed= torch.zeros(self.total_nb_graphs).to(self.device)    
        node_features=torch.zeros(self.total_nb_graphs,self.card.max(),self.nb_feature).to(self.device)    
        #print('weighs:',self.weighs.device,'device:',self.device,'card:',self.card.device,'A:',self.A.device,'labels:',self.labels.device)
        
        for k in range(len(input)):            
            g1=input[k][0]
            g2=input[k][1]
            
            n=self.card[g1]
            m=self.card[g2]
            if not f_computed[g1]:
                node_features[g1,0:n,:]=self.gcn(self.labels[g1][0:n,:],self.A[g1][0:n*n].view(n,n)+torch.eye(n,n,device=self.device))                
                f_computed[g1]=1
                #print('m1=',self.gcn(self.labels[g1][0:n,:],self.A[g1][0:n*n].view(n,n)+torch.eye(n,n,device=self.device)))
            if not f_computed[g2]:
                node_features[g2,0:m,:]=self.gcn(self.labels[g2][0:m,:],self.A[g2][0:m*m].view(m,m)+torch.eye(m,m,device=self.device))
                f_computed[g2]=1
                #print('m2=',self.gcn(self.labels[g2][0:m,:],self.A[g2][0:m*m].view(m,m)+torch.eye(m,m,device=self.device)))
        
        for k in range(len(input)):            
            g1=input[k][0]
            g2=input[k][1]
            
            n=self.card[g1]
            m=self.card[g2]
            m1=node_features[g1,0:n,:]
            m2=node_features[g2,0:m,:]
            C=self.construct_cost_matrix(g1,g2,m1,m2,node_subst,edge_costs,nodeInsDel,edgeInsDel)      
            S=self.mapping_from_cost(C,n,m) # FW
            #S=self.mapping_from_similarity(C,n,m) # SVD or iterated power.
            #M=self.similarity_from_cost(C)
            #S=self.mapping_from_similarity(M,n+1,m+1)
            #S=torch.exp(S) # enforce the difference, accelerate the convergence. 
            #S=self.eps_assigment_from_mapping(S)
            v=torch.flatten(S)
            
            normalize_factor=1.0
            if self.normalize:
                nb_edge1=(self.A[g1][0:n*n] != torch.zeros(n*n,device=self.device)).int().sum()
                nb_edge2=(self.A[g2][0:m*m] != torch.zeros(m*m,device=self.device)).int().sum()
                #normalize_factor=(nodeInsDel*n+edgeInsDel*nb_edge1)*(nodeInsDel*m+edgeInsDel*nb_edge2)                                    
                normalize_factor=(nodeInsDel*(n+m)+edgeInsDel*(nb_edge1+nb_edge2))
            ged[k]=.5*(v.t()@(C+torch.diag(C))@v)/normalize_factor    
        return ged
    
    def from_weighs_to_costs(self):
        
        #cn=torch.exp(self.node_weighs)
        #ce=torch.exp(self.edge_weighs)
        cn=self.node_weighs*self.node_weighs
        #cn=torch.where(x<1.0,x,torch.ones_like(x))
        ce=self.edge_weighs*self.edge_weighs
        #ce=torch.where(y<1.0,y,torch.ones_like(y))
        total_cost=cn.sum()+ce.sum()
        cn=cn/total_cost
        ce=ce/total_cost
        edgeInsDel=ce[-1]

        #node_costs=torch.zeros((self.nb_labels,self.nb_labels),device=self.device)
        #upper_part=torch.triu_indices(node_costs.shape[0],node_costs.shape[1],offset=1,device=self.device)        
        #node_costs[upper_part[0],upper_part[1]]=cn[0:-1]
        #node_costs=node_costs+node_costs.T

        if self.nb_edge_labels>1:
            edge_costs=torch.zeros((self.nb_edge_labels,self.nb_edge_labels),device=self.device)
            upper_part=torch.triu_indices(edge_costs.shape[0],edge_costs.shape[1],offset=1,device=self.device)        
            edge_costs[upper_part[0],upper_part[1]]=ce[0:-1]
            edge_costs=edge_costs+edge_costs.T
        else:
            edge_costs=torch.zeros(0,device=self.device)
        
        return cn[0],cn[1],edge_costs,edgeInsDel
    
    def build_node_dictionnary(self,GraphList):
        #extraction de tous les labels d'atomes
        node_labels=[]
        for G in Gs:
            for v in nx.nodes(G):
                if not G.nodes[v]['label'][0] in node_labels:
                    node_labels.append(G.nodes[v]['label'][0])
        node_labels.sort()
        #extraction d'un dictionnaire permettant de numéroter chaque label par un numéro.
        dict={}
        k=0
        for label in node_labels:
            dict[label]=k
            k=k+1
        print(node_labels)
        print(dict,len(dict))
    
        return dict,max(max([[int(G[e[0]][e[1]]['bond_type']) for e in G.edges()] for G in GraphList]))
    
    def from_networkx_to_tensor(self,G,dict,nb_max_vertices):    
        A=torch.tensor(nx.to_scipy_sparse_matrix(G,dtype=float,weight='bond_type').todense(),dtype=torch.int)        
        lab=torch.zeros(nb_max_vertices,self.nb_labels)
        k=0
        
        for v in nx.nodes(G):
            lab[k][dict[G.nodes[v]['label'][0]]]=1.0
            k=k+1
   
        return (A.view(1,A.shape[0]*A.shape[1]),lab)

    def construct_cost_matrix(self,g1,g2,m1,m2,node_subst,edge_costs,nodeInsDel,edgeInsDel):
        n = self.card[g1].item()
        m = self.card[g2].item()
        
        A1=torch.zeros((n+1,n+1),dtype=torch.int,device=self.device)
        A1[0:n,0:n]=self.A[g1][0:n*n].view(n,n)
        A2=torch.zeros((m+1,m+1),dtype=torch.int,device=self.device)
        A2[0:m,0:m]=self.A[g2][0:m*m].view(m,m)
        
        
         # costs: 0 node subs, 1 nodeIns/Del, 2 : edgeSubs, 3 edgeIns/Del
        
        #C=cost[3]*torch.cat([torch.cat([C12[l][k] for k in range(n+1)],1) for l in range(n+1)])
        #Pas bien sur mais cela semble fonctionner.
        C=edgeInsDel*self.matrix_edgeInsDel(A1,A2)
        if self.nb_edge_labels>1:
            for k in range(self.nb_edge_labels):
                for l in range(self.nb_edge_labels):
                    if k != l:
#                    C.add_(self.matrix_edgeSubst(A1,A2,k+1,l+1),alpha=edge_costs[k][l])
                        C=C+edge_costs[k][l]*self.matrix_edgeSubst(A1,A2,k+1,l+1)
        #C=cost[3]*torch.tensor(np.array([ [  k!=l and A1[k//(m+1),l//(m+1)]^A2[k%(m+1),l%(m+1)] for k in range((n+1)*(m+1))] for l in range((n+1)*(m+1))]),device=self.device)        

        #l1=self.labels[g1][0:n,:]
        #l2=self.labels[g2][0:m,:]        
        D=torch.zeros((n+1)*(m+1),device=self.device)
        D[n*(m+1):]=nodeInsDel
        D[n*(m+1)+m]=0
        norm_l1=torch.einsum('i,j->ij',torch.diag(m1@m1.T),torch.ones(m,device=self.device))
        norm_l2=torch.einsum('i,j->ij',torch.diag(m2@m2.T),torch.ones(n,device=self.device)).T
        scalar=m1@m2.T
        
        d=torch.empty(n,m+1,device=self.device)
        x=(norm_l1+norm_l2-2*scalar)/m1.shape[1]
        eps=10**(-6)
        d[:,0:m]=node_subst*torch.sqrt(torch.where(x>eps,x,eps*torch.ones(n,m,device=self.device)))       
        d[:,-1]=nodeInsDel
       # print('d=',d)
        D[0:n*(m+1)]=d.view(1,d.shape[0]*d.shape[1])
        #D[[i*(m+1)+m for i in range(n)]]=nodeInsDel
        #pdist = nn.PairwiseDistance(p=2)
        #D[[k for k in range(n*(m+1)) if k%(m+1) != m]]=torch.tensor([pdist(l1[k//(m+1),:],l2[k%(m+1),:]) for k in range(n*(m+1)) if k%(m+1) != m],device=self.device )
        mask = torch.diag(torch.ones_like(D))
        C=mask*torch.diag(D) + (1. - mask)*C
        
        #C[range(len(C)),range(len(C))]=D
      
        return C
    def matrix_edgeInsDel(self,A1,A2):
        Abin1=(A1!=torch.zeros((A1.shape[0],A1.shape[1]),device=self.device))
        Abin2=(A2!=torch.zeros((A2.shape[0],A2.shape[1]),device=self.device))
        C1=torch.einsum('ij,kl->ijkl',torch.logical_not(Abin1),Abin2)
        C2=torch.einsum('ij,kl->ijkl',Abin1,torch.logical_not(Abin2))
        C12=torch.logical_or(C1,C2).int()
    
        return torch.cat(torch.unbind(torch.cat(torch.unbind(C12,1),1),0),1)

    def matrix_edgeSubst(self,A1,A2,lab1,lab2):
        Abin1=(A1==lab1*torch.ones((A1.shape[0],A1.shape[1]),device=self.device)).int()
        Abin2=(A2==lab2*torch.ones((A2.shape[0],A2.shape[1]),device=self.device)).int()
        C=torch.einsum('ij,kl->ijkl',Abin1,Abin2)
        
        return torch.cat(torch.unbind(torch.cat(torch.unbind(C,1),1),0),1)
    
    def mapping_from_cost(self,C,n,m):
        c=torch.diag(C)
        D=C-torch.eye(C.shape[0],device=self.device)*c
        x0=svd.eps_assigment_from_mapping(torch.exp(-c.view(n+1,m+1)),10).view((n+1)*(m+1),1)
    
        return svd.franck_wolfe(x0,D,c,2.0,10,n,m)
    
    def similarity_from_cost(self,C):
        N=C.shape[0]
        
     
        return (C.max()*torch.eye(N,device=self.device) -C)
    
    def mapping_from_similarity(self,C,n,m):
        M=self.similarity_from_cost(C)
        first_ev=compute_major_axis(M)
        #first_ev=self.iterated_power(M,inv=True)
        if(first_ev.sum() <0):
            first_ev=-first_ev
        S=torch.exp(first_ev.view(n+1,m+1)) # enforce the difference, accelerate the convergence. 
        S=self.eps_assigment_from_mapping(S)
        return  S
        
    def eps_assigment_from_mapping(self,S):
        ones_n = torch.ones(S.shape[0],device=S.device)
        ones_m = torch.ones(S.shape[1],device=S.device)
    
        Sk = S
        for i in range(20):
            D=torch.diag(1.0/(Sk@ones_m))
            D[D.shape[0]-1,D.shape[1]-1]=1.0
            Sk1 = D@Sk
            D=torch.diag(1.0/(ones_n@Sk1))
            D[D.shape[0]-1,D.shape[1]-1]=1.0
            Sk = Sk1@D
        
        return Sk



In [5]:
from regression import KnnRegressFromGED as regress
from regression import train_set_for_knnregression as create_dataset

nb=len(Gs)
#class1=torch.tensor([k for k in range(len(y)) if y[k]==1])
#class2=torch.tensor([k for k in range(len(y)) if y[k]==0])

train_size=70
nb_test=10
train_graphs=torch.randperm(nb)[0:train_size]
print('train graphs:', train_graphs)



#couples=[]
#for k in range(nb_elt):
#    if (y[data[k][0]]!=y[data[k][1]]):
#        yt[k]=-1.0        
#data,yt=train_set_for_kernel_ridge(train_graphs,y,train_size)
data,yt=create_dataset(train_graphs,y,train_size,nb_test)
print('data=',data,data.shape[0])
print('yt=',yt)
#print(couples[1:2])

torch.cuda.empty_cache()
if torch.cuda.is_available():
    device = torch.device("cuda:0")          # a CUDA device object    


train graphs: tensor([ 42,  57, 121, 141,  59,  92, 143,  68, 142, 129,  65,   1,  35,  17,
        152, 158, 111,  24, 154,   3,  89,   7,  18, 138, 132,  23,  28,  71,
         30, 120, 148,  14,  33, 124,  15,  67,  19, 125,  63,  12, 100,  80,
         49, 110, 151,  82, 131, 115,  21, 102, 156,  29, 106,   9,  53, 127,
         87,  66,   6,  16,  13,  36,  47,  41,  81,  83,  69,  56, 144, 126])
data= tensor([[ 13,  42],
        [ 13,  57],
        [ 13, 121],
        ...,
        [126,  66],
        [126,   6],
        [126,  16]], dtype=torch.int32) 600
yt= tensor([ 82.0000, 144.2000, 148.5000, 126.0000, 132.0000, 171.0000, 163.0000,
        181.0000, 164.0000, 151.5000, 137.0000,  37.3000, 101.5000,  84.4000,
        156.5000, 174.0000, 151.0000,  59.0000, 173.2000,  10.8000, 157.5000,
        135.0000, 156.0000, 159.5000, 139.0000,  52.5000, 104.5000, 186.0000,
         92.0000, 178.0000, 218.0000,  84.7000, 112.5000, 167.0000,  95.5000,
        177.2000, 166.0000, 195.0000, 

In [6]:
from collections import OrderedDict
from triangular_losses import ReducedTriangularConstraint as triangular_constraint

def regression(model,data,yt,nb_iter):
    torch.autograd.set_detect_anomaly(True)
    input=data.to(device)
    target=yt.to(device) 
    
    criterion = torch.nn.MSELoss()
    criterionTri=triangular_constraint()
    
        
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
    print(device)

#torch.cat((same_class[0:20],diff_class[0:20]),0).to(device)
    
#torch.ones(40,device=device)
#target[20:]=-1.0
#target=(yt[0:20]).to(device)
    InsDel=np.empty((nb_iter,2))
    nodeSub=np.empty(nb_iter)
    node_subst,nodeInsDel,edge_costs,edgeInsDel=model[0].from_weighs_to_costs()
    edgeSub=np.empty((nb_iter,int(edge_costs.shape[0]*(edge_costs.shape[0]-1)/2)))
    loss_plt=np.empty(nb_iter)
    #print('alpha=',regress.coef_dist*regress.coef_dist*10.0)
    
    
    for t in range(nb_iter):  
        
        # Forward pass: Compute predicted y by passing data to the model    
        node_subst,nodeInsDel,edge_costs,edgeInsDel=model[0].from_weighs_to_costs()

        y_pred = model(input).to(device)
            
        # Compute and print loss
        loss = criterion(y_pred, target[-nb_test:]).to(device)                                                           
        triangularInq=criterionTri(node_subst,nodeInsDel,edge_costs,edgeInsDel)        
        loss=loss*(1+triangularInq)
        loss.to(device)
        
        InsDel[t][0]=nodeInsDel.item()
        InsDel[t][1]=edgeInsDel.item()
        rmse=np.sqrt(loss.item()/nb_test)
        loss_plt[t]=rmse
        nodeSub[t]=node_subst.item()
        if edge_costs.shape[0] > 0:
            k=0
            for p in range(edge_costs.shape[0]):
                for q in range(p+1,edge_costs.shape[0]):
                    edgeSub[t][k]=edge_costs[p][q]
                    k=k+1
        if t % 20 == 0 or t==0:
                            
            #print('Distances: ',y_pred)
            print('alpha=',model[1].coef_dist*model[1].coef_dist)
            #print('lambda=',model[1].regu*model[1].regu)
            print('target=',target[-nb_test:])
            print('y_pred=',y_pred)
            #print('Loss Triangular:',triangularInq.item())
            print('node subst. cost :',node_subst.item())            
            print('nodeInsDel:',nodeInsDel.item())
            if edge_costs.shape[0]>0:
                print('edge_costs :')
                print(edge_costs)
            print('edgeInsDel:',edgeInsDel.item())
        
        
            # Zero gradients, perform a backward pass, and update the weights.
        print(t, rmse)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    
    return InsDel,nodeSub,edgeSub,loss_plt

nb_test=10
model = nn.Sequential( OrderedDict([
        ('ComputeDist',Net(Gs,normalize=True,device=device)),
        ('RegressFromDist',regress(yt.to(device),10,nb_test=nb_test,device=device,weights='distance'))
        ])) 
print('Parameters:')
for param in model.parameters():
    print(type(param), param.size())
    print(param)
    
#with torch.autograd.profiler.profile(use_cuda=True) as prof:    
InsDel, nodeSub,edgeSub,loss_plt=regression(model,data,yt,200)


['C', 'O', 'S']
{'C': 0, 'O': 1, 'S': 2} 3
nb_edge labels= 1
torch.Size([164, 121])
adjacency matrices tensor([[0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 1.,  ..., 1., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 1., 0.]], device='cuda:0')
node labels tensor([[[1., 0., 0.],
         [1., 0., 0.],
         [0., 1., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[1., 0., 0.],
         [1., 0., 0.],
         [0., 0., 1.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[1., 0., 0.],
         [1., 0., 0.],
         [0., 0., 1.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        ...,

        [[1., 0., 0.],
         [1., 0., 0.],
         [1., 0., 0.],
         ...,
         [1., 0., 0.],
         [1., 0., 0.],
         

KeyboardInterrupt: 

In [ ]:

plt.figure(0)
plt.plot(InsDel[0:500,0],label="node")
plt.plot(InsDel[0:500,1],label="edge")
plt.title('Node/Edge insertion/deletion costs')
plt.legend()
plt.figure(1)
for k in range(nodeSub.shape[1]):
    plt.plot(nodeSub[0:500,k])
plt.title('Node Substitutions costs')
plt.figure(2)
plt.plot(loss_plt[0:500])
for k in range(edgeSub.shape[1]):
    plt.plot(edgeSub[0:500,k])
plt.title('Loss')
plt.show()
plt.close()